# 0. Install and Import Dependencies

In [ ]:
# !pip install mediapipe opencv-python pandas

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose # initiate model pose

# 1. Extract Video to Dataset

In [ ]:
# for initiate header
landmarks = ["class"]
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
# writing header for dataset file
with open('../dataset/momentum_v1.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
# function to write to csv file
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        with open('../dataset/momentum_1v.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [ ]:
vid_path = "../../data/momentum/?" # path to video
cap = cv2.VideoCapture(vid_path) #initiate video capture

#Initiate Holistic Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    # streaming the video
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        # Make detections
        results = pose.process(image)
    
        # Recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        if results.pose_landmarks:
            import matplotlib.pyplot as plt

            fig = plt.figure()
            ax = fig.add_subplot(111, projection="3d")
            fig.subplots_adjust(left=0.0, right=1, bottom=0, top=1)

            plot_world_landmarks(
                plt,
                ax,
                results.pose_world_landmarks,
            )

        # # extract landmarks
        # k = cv2.waitKey(1)
        # if k == 106: # while j is pressed, landmark will be recorded as class 0 => normal
        #     export_landmark(results, 0)
        # if k == 107: # while k is pressed, landmark will be recorded as class 1 => narrow
        #     export_landmark(results, 1)
        # if k == 108: # while l is pressed, landmark will be recorded as class 2 => wide
        #     export_landmark(results, 2)
            
        # Stream vid result
        cv2.imshow("Raw Cam Feed", image)

        # press 'q' than the video will stop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
result